In [1]:
import random
import sys
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics.cluster import adjusted_rand_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scripts.data_generator as data_generator
import scripts.feature_ranking as feature_ranking
import scripts.features_2d as features_2d
import scripts.ga as ga
import scripts.preprocess as preprocess
import scripts.ga_evaluation as ga_evaluation
import scripts.bio_analysis as bio_analysis
import tensorflow as tf
from IPython import get_ipython
from keras.backend.tensorflow_backend import set_session
from tqdm import tqdm
from collections import Counter
import seaborn as sns
import time

config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default

plt.ion()
plt.show()
sys.path.append("..")

random_state=1
random.seed( random_state )
np.random.seed(random_state)


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.datasets.samples_generator module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5



In [2]:
%load_ext autoreload
%autoreload 2

# Steps
- Clustering 1d to select best discriminant features

- Clustering 2d to select redundant, close and outlier features

In [3]:
random_state=0
random.seed( random_state )
np.random.seed(random_state)
results = None

In [5]:
filenames = np.array(["khan","west",  "gordon",'borovecki'])
clustering = "gmm"
path = 'data/microarray/'
method = "adapted_ratkowsky_lance"
imp_f = np.arange(20)
for name in filenames:
    t1 = time.time()
    data = pd.read_pickle(f'{path}' + name + '.pkl')
    truth = data["truth"].values
    data = data.drop("truth", axis = 1).values


    n_clusters = len(np.unique(truth))

    z_file= f"data/microarray/Z_{name}_correlation.npy"
    print(f"\n##########  {name}, {data.shape}")

    # Clustering 1D
    meta_features = feature_ranking.rank_features(data,
                                              nb_bins=20,
                                              rank_threshold=90,
                                              z_file=z_file,
                                              metric='correlation',
                                              redundant_threshold=0.4)
    t2 = time.time()
    print(f"TIME: 1d Features : {(t2-t1)/60} min")

    model_file = f'models/gmm_arl.h5' 

    population, n = features_2d.run(data,
                                n_clusters,
                                meta_features,
                                model_file=model_file,
                                add_close_population=False)
    t3 = time.time()
    t4 = time.time()
    print(f"TIME: 2d scores: {(t4-t3)/60} min")
    round_size = 3
    epochs = 10*round_size

    sampling = {
        "ARCHIVE2D": { 
            "ga": 0.25,
            "max": 0.25 },
        "CLOSE": { 
            "ga": 0.4,
            "max": 0.4 },
        "IMP1D": { 
            "ga": 0.25,
            "max": 0.25 },
        "RANDOM": { 
            "ga": 0.1,
            "max": 0.1},
        }
    params = ga.ga_parameters(
        n_clusters,
        data.shape[1],
        truth,
        meta_features,
        method=method,
        truth_methods=['ari'],
        archive_2d=population[:data.shape[1] // 4],
        epochs=epochs,
        sampling = sampling,
        round_size=round_size,
        allow_subspace_overlap = True,
        improvement_per_mutation_report = False,
        clustering = clustering
        
    )
    solutions, archive= ga.run(data, params)
#     display(params["report"].groupby(["op", "improvement"]).count())
    solutions["dataset_name"] = name
    
    t5 = time.time()
    n_total = t5-t1
    print(f"TIME: GA: {(t5-t4)/60} min")
    print(f"TIME: Total: {(t5-t1)/60} min")
    solutions["total_time"] = round((t5-t1)/60, 1)
    solutions["t(feature_sel)"] = round((t2-t1)/60, 1)
    solutions["t(cnn)"] = round((t3-t2)/60, 1)
    solutions["t(clustering2d)"] = round((t4-t3)/60, 1)
    solutions["t(ga)"] = round((t5-t4)/60, 1)
    
    solutions["n_ga"] = archive.shape[0]
    solutions["n_cnn"] = n_total
    solutions["input_size"] = data.shape[1]
    
    
    solutions["n_non_redundant"] = meta_features[(meta_features["redundant"] ==1) ].shape[0]
    solutions["n_imp"] = meta_features[(meta_features["relevance"] !=0)].shape[0]
    solutions["n_imp4"] = meta_features[(meta_features["relevance"] ==4)].shape[0]
    solutions["n_imp3"] = meta_features[(meta_features["relevance"] ==3)].shape[0]
    
    
    if results is None: 
        results = solutions
    else:
        results = pd.concat([results, solutions], ignore_index = True)
    results.to_excel("reports/microarray_gmm.xlsx")
    


##########  khan, (63, 2308)
*** Computing 1D feature ranking ...
Dispersion tests took 0.02 sec
Entropy computation 0.35 sec
KNN computation 0.18 sec
Sorting and thresholds 0.02 sec
Loading clustering from file
Hierarchical clustering 0.01 sec
Handle redundant features 0.63 sec
Returning 381 redundant features and  83 important features
TIME: 1d Features : 0.020270029703776043 min
*** Exploring 2D feature space with NN ...
trimming 1665 to 963
handle_close_important (963, 3), total 1678, 0.16005289554595947
relevant_features 606 => computing 3586 
trimming 3123 to 381
handle_important_features (1344, 3),  total 3586, 0.16005289554595947
irrelevant_features 1321 => computing 3953
trimming 3042 to 963
handle_not_important_features (2307, 3), total 3953, 0.16005289554595947
handle_all_features 1927 => computing 5772


  0%|          | 0/31 [00:00<?, ?it/s]

trimming 4666 to 1927
handle_all_features (4234, 3),  total 5772, 0.16005289554595947
Returning (4234, 3), explored a total of 14989 feature pairs
TIME: 2d scores: 1.1920928955078126e-08 min
*** Optimization algorithm 
Non redundant features 1927, orig size 2308, nb imp : 83
Selecting (50, 4) from archive


 10%|▉         | 3/31 [00:08<01:21,  2.93s/it]

,adapted_ratkowsky_lance,ari,features,partition,size,structure
0,0.161918,0.06,"[15, 35, 60, 66, 91, 112, 118, 148, 159, 258, ...","[3, 3, 3, 0, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 2, ...",95.0,"Counter({1: 27, 2: 14, 3: 13, 0: 9})"


...

 97%|█████████▋| 30/31 [04:27<00:08,  8.05s/it]

,adapted_ratkowsky_lance,ari,features,partition,size,structure
0,0.338797,-0.03,"[584, 724, 1036, 1073, 2960, 3060, 3175, 3325,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ...",80.0,"Counter({1: 22, 0: 9})"
1,0.331147,0.54,"[2223, 2697, 2840, 2844, 2938, 2965, 2983, 298...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...",163.0,"Counter({1: 18, 0: 13})"
2,0.302408,-0.01,"[4297, 5424, 8814, 9834, 12350, 13852, 13874, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",11.0,"Counter({0: 27, 1: 4})"
3,0.306130,0.03,"[106, 12273, 12759, 13366, 16318, 19716]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",6.0,"Counter({0: 27, 1: 4})"
4,0.293582,0.03,"[1140, 1451, 1875, 2064, 3432, 5760, 7451, 826...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",19.0,"Counter({0: 27, 1: 4})"
5,0.316202,0.03,"[465, 1396, 2149, 2286, 3232, 4794, 5484, 9286...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",15.0,"Counter({0: 27, 1: 4})"
6,0.314743,0.10,"[1748, 3721, 4597, 7009, 7430, 7431, 7434, 744...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ...",32.0,"Counter({1: 17, 0: 14})"
7,0.281488,-0.03,"[370, 1136, 1703, 1804]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, ...",4.0,"Counter({1: 23, 0: 8})"
8,0.331061,0.54,"[812, 1001, 2223, 2844, 2985, 3028, 3126, 3129...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...",163.0,"Counter({1: 18, 0: 13})"
9,0.277822,0.15,"[62, 954, 1553, 7611, 8811, 13258, 15107, 1957...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",11.0,"Counter({0: 24, 1: 7})"


Selecting (4, 4) from archive


100%|██████████| 31/31 [04:40<00:00,  9.05s/it]


TIME: GA: 4.675550969441732 min
TIME: Total: 7.306553395589193 min


In [6]:
results.groupby("dataset_name").agg({"ari": max})

,ari
dataset_name,
borovecki,0.54
gordon,0.86
khan,0.90
west,0.03


# Supervised analysis of datasets

In [ ]:
import scripts.ga_evaluation as ga_evaluation
filenames = np.array(["khan","west",  "gordon",'borovecki'])
path = 'data/microarray/'
imp_f = np.arange(20)
result_df = pd.DataFrame()
for name in filenames:
    t1 = time.time()
    data = pd.read_pickle(f'{path}' + name + '.pkl')
    truth = data["truth"].values
    data = data.drop("truth", axis = 1).values

    n_clusters = len(np.unique(truth))

    result = {"Dataset" : name,
             "Original Dimensions": f"{data.shape[0]} x {data.shape[1]}",
             "Cluster sizes" : ", ".join(np.array(list(Counter(truth).values())).astype(str))}
    
    predK = KMeans(n_clusters= n_clusters, random_state = 2).fit(data).labels_
    ari_all = adjusted_rand_score(truth, predK)
    
    predK = KMeans(n_clusters= n_clusters, random_state = 2).fit(data[:, :5]).labels_
    ari_top10 = adjusted_rand_score(truth, predK)
    
    pca = PCA(2)
    pca_data = pca.fit_transform(data)

    predK = KMeans(n_clusters= n_clusters, random_state = 2).fit(pca_data).labels_
    ari_pca = adjusted_rand_score(truth, predK)
    
    r1 = ga_evaluation.random_sampling(data, truth, n_clusters, algo = "gmm")
    r2 = ga_evaluation.random_sampling(data, truth, n_clusters, algo = "hdbscan")
    result["ARI all dataset"] = round(ari_all,2)
    result["ARI PCA dataset"] = round(ari_pca,2)
    result["ARI top 10 features"] = round(ari_top10,2)
    result["Random GMM"] = round(r1,2)
    result["Random HDBSCAN"] = round(r2,2)
    result_df = result_df.append(result, ignore_index = True)

result_df

# Run time analysis

In [ ]:
results = pd.read_excel("reports/r_cnn.xlsx", index_col=0)

In [ ]:
results["label"] = results["dataset_name"] + " (" +results["input_size"].astype(str) + " features) " 

In [ ]:
perf = results.groupby("input_size").min()[['t(feature_sel)', 't(cnn)','t(ga)']]

In [ ]:
perf = perf.rename(columns = {
    't(feature_sel)' : '1D Feature ranking', 
    't(cnn)' : '2D Feature ranking with NN',
    't(ga)': 'Optimization algorithm for top 10 subspaces'
})

In [ ]:
plt.figure(figsize = (10,3))
ax = plt.gca()
perf.plot(kind='bar', stacked=True, ax = ax)
plt.ylabel("time (min)")
plt.xticks(rotation = 0)
sns.despine()
plt.title("Run times on West, Khan, Gordon and Boroveki datasets")
plt.xlabel("Number of dimensions in the input dataset")
plt.savefig(f"images/run_times.pdf", bbox_inches='tight')

In [7]:
from sklearn import mixture
import hdbscan

# Best scores using supervised feature selection

In [9]:
import scripts.ga_evaluation as ga_evaluation
filenames = np.array(["khan","west",  "gordon",'borovecki'])
path = 'data/microarray/'

In [ ]:
for name in filenames:
    t1 = time.time()
    data = pd.read_pickle(f'{path}' + name + '.pkl')
    truth = data["truth"].values
    data = data.drop("truth", axis = 1).values
    print(Counter(truth))
    n_clusters = len(np.unique(truth))
    gmm_scores = []
    hdbscan_scores = []
    for i in range(2, 50):
        input_data = data[:, :i]
        gmm = mixture.GaussianMixture(n_components=n_clusters,
                          covariance_type="full", random_state=0)
        pred = gmm.fit_predict(input_data)
        ari = adjusted_rand_score(truth, pred)
        gmm_scores.append(ari)

        pred = hdbscan.HDBSCAN(min_cluster_size =2).fit(input_data).labels_
        ari = adjusted_rand_score(truth, pred)
        hdbscan_scores.append(ari)

        
    print(f"\n\n\n{name} GMM ari = {max(gmm_scores)}, ")
    print(f"{name} HDBSCAN ari = {max(hdbscan_scores)}, ")
    

# Mutual Information

In [12]:
from sklearn.feature_selection import chi2,  mutual_info_classif, SelectKBest

In [13]:
for name in filenames:
    t1 = time.time()
    data = pd.read_pickle(f'{path}' + name + '.pkl')
    truth = data["truth"].values
    data = data.drop("truth", axis = 1).values
    print(Counter(truth))
    n_clusters = len(np.unique(truth))
    gmm_scores = []
    hdbscan_scores = []
    sel = SelectKBest(mutual_info_classif, k=50).fit_transform(data, truth)
    for i in range(2, 50):
        input_data = sel[:, :i]
        gmm = mixture.GaussianMixture(n_components=n_clusters,
                          covariance_type="full", random_state=0)
        pred = gmm.fit_predict(input_data)
        ari = adjusted_rand_score(truth, pred)
        gmm_scores.append(ari)

        pred = hdbscan.HDBSCAN(min_cluster_size =2).fit(input_data).labels_
        ari = adjusted_rand_score(truth, pred)
        hdbscan_scores.append(ari)

        
    print(f"\n\n\n{name} GMM ari = {max(gmm_scores)}, ")
    print(f"{name} HDBSCAN ari = {max(hdbscan_scores)}, ")

Counter({1: 23, 3: 20, 2: 12, 0: 8})



khan GMM ari = 1.0, 
khan HDBSCAN ari = 0.9753291540426958, 
Counter({0: 25, 1: 24})



west GMM ari = 0.7652246371779737, 
west HDBSCAN ari = 0.3423504867872045, 
Counter({0: 150, 1: 31})



gordon GMM ari = 0.9731180281933411, 
gordon HDBSCAN ari = 0.9060143786845584, 
Counter({1: 17, 0: 14})



borovecki GMM ari = 1.0, 
borovecki HDBSCAN ari = 0.775027912169706, 


In [ ]:
import scripts.ga_evaluation as ga_evaluation
filenames = np.array(["khan","west",  "gordon",'borovecki'])
path = 'data/microarray/'

for name in filenames:
    t1 = time.time()
    data = pd.read_pickle(f'{path}' + name + '.pkl')
    truth = data["truth"].values
    input_data = data.drop("truth", axis = 1).values

    n_clusters = len(np.unique(truth))
    gmm = mixture.GaussianMixture(n_components=n_clusters,
                      covariance_type="full", random_state=0)
    pred = gmm.fit_predict(input_data)
    ari = adjusted_rand_score(truth, pred)
    print(f"{name} GMM ari = {ari}")

    pred = hdbscan.HDBSCAN(min_cluster_size =2).fit(input_data).labels_
    ari = adjusted_rand_score(truth, pred)
    print(f"{name} HDBSCAN ari = {ari}")

In [ ]:
import scripts.ga_evaluation as ga_evaluation
filenames = np.array(['borovecki'])
path = 'data/microarray/'

for name in filenames:
    t1 = time.time()
    data = pd.read_pickle(f'{path}' + name + '.pkl')
    truth = data["truth"].values
    input_data = data.drop("truth", axis = 1).values
    input_data = input_data[:, :10000]
    

    n_clusters = len(np.unique(truth))
    
    pred = hdbscan.HDBSCAN(min_cluster_size =2).fit(input_data).labels_
    ari = adjusted_rand_score(truth, pred)
    print(f"{name} HDBSCAN ari = {ari}")
    
    gmm = mixture.GaussianMixture(n_components=n_clusters,
                      covariance_type="full", random_state=0)
    pred = gmm.fit_predict(input_data)
    ari = adjusted_rand_score(truth, pred)
    print(f"{name} GMM ari = {ari}")




# Execution time analysis


In [ ]:
results = pd.read_excel("reports/r_cnn.xlsx", index_col=0)

results["label"] = results["dataset_name"] + " (" +results["input_size"].astype(str) + " features) " 

perf = results.groupby("input_size").min()[['t(feature_sel)', 't(cnn)','t(ga)']]

perf = perf.rename(columns = {
    't(feature_sel)' : '1D Feature ranking', 
    't(cnn)' : '2D Feature ranking with NN',
    't(ga)': 'Optimization algorithm for top 10 subspaces'
})



plt.figure(figsize = (10,3))
ax = plt.gca()
perf.plot(kind='bar', stacked=True, ax = ax)
plt.ylabel("time (min)")
plt.xticks(rotation = 0)
sns.despine()
plt.title("Run times on West, Khan, Gordon and Boroveki datasets")
plt.xlabel("Number of dimensions in the input dataset")
plt.savefig(f"images/run_times.pdf", bbox_inches='tight')